# Monte Carlo Methods

In this notebook, you will write your own implementations of many Monte Carlo (MC) algorithms. 

While we have provided some starter code, you are welcome to erase these hints and write your code from scratch.

### Part 0: Explore BlackjackEnv

We begin by importing the necessary packages.

In [1]:
import sys
import gym
import numpy as np 
from collections import defaultdict

from plot_utils import plot_policy ,plot_blackjack_values

Use the code cell below to create an instance of the [Blackjack](https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py) environment.

In [2]:
env = gym.make('Blackjack-v0')

ImportError: DLL load failed: The specified procedure could not be found.

Each state is a 3-tuple of:
- the player's current sum $\in \{0, 1, \ldots, 31\}$,
- the dealer's face up card $\in \{1, \ldots, 10\}$, and
- whether or not the player has a usable ace (`no` $=0$, `yes` $=1$).

The agent has two potential actions:

```
    STICK = 0
    HIT = 1
```
Verify this by running the code cell below.

In [4]:
print(env.observation_space)
print(env.action_space)

NameError: name 'env' is not defined

Execute the code cell below to play Blackjack with a random policy.  

(_The code currently plays Blackjack three times - feel free to change this number, or to run the cell multiple times.  The cell is designed for you to get some experience with the output that is returned as the agent interacts with the environment._)

In [ ]:
for i in range(3):
    state = evn.reset()
    while True:
        print(state)
        action = env.action_space.sample()
        state,reward,done,_ = env.step(action)
        if done:
            print(f'End game ! Reward : {reward}')
            print('You Won :)' )if reward > 0 else print('You Lost :( \n')
            break

### Part 1: MC Prediction

In this section, you will write your own implementation of MC prediction (for estimating the action-value function).  

We will begin by investigating a policy where the player _almost_ always sticks if the sum of her cards exceeds 18.  In particular, she selects action `STICK` with 80% probability if the sum is greater than 18; and, if the sum is 18 or below, she selects action `HIT` with 80% probability.  The function `generate_episode_from_limit_stochastic` samples an episode using this policy. 

The function accepts as **input**:
- `bj_env`: This is an instance of OpenAI Gym's Blackjack environment.

It returns as **output**:
- `episode`: This is a list of (state, action, reward) tuples (of tuples) and corresponds to $(S_0, A_0, R_1, \ldots, S_{T-1}, A_{T-1}, R_{T})$, where $T$ is the final time step.  In particular, `episode[i]` returns $(S_i, A_i, R_{i+1})$, and `episode[i][0]`, `episode[i][1]`, and `episode[i][2]` return $S_i$, $A_i$, and $R_{i+1}$, respectively.

In [ ]:
def generate_episode_from_limit_stochastic(bj_env):
    episode = []
    state = bj_env.reset()
    while True:
        prob = [0.8,0.2] if state[0] > 18 else [0.2,0.8]
        action = np.random.choice(np.arange(2) , p = prob)
        next_state,reward , done ,info = bj_env.step(action)
        episode.append((state,action,reward))
        state = next_state
        if done :
            break
            
        return episode

Execute the code cell below to play Blackjack with the policy. 

(*The code currently plays Blackjack three times - feel free to change this number, or to run the cell multiple times.  The cell is designed for you to gain some familiarity with the output of the `generate_episode_from_limit_stochastic` function.*)

In [ ]:
for i in range(3):
    print(generate_episode_from_limit_stochastic(env))

Now, you are ready to write your own implementation of MC prediction.  Feel free to implement either first-visit or every-visit MC prediction; in the case of the Blackjack environment, the techniques are equivalent.

Your algorithm has three arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `generate_episode`: This is a function that returns an episode of interaction.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.

In [6]:
def mc_prediction_q (env , num_episodes , generate_episode , gamma = 1.0):
    return_sum =defaultdict(lambda: np.zero(env.action_space.n))
    N =defaultdict(lambda: np.zero(env.action_space.n))
    Q =defaultdict(lambda: np.zero(env.action_space.n))
    
    for i in range( 1, num_episodes + 1):
        if i % 1000:
            print(f'\rEpisode {i} / {num_episodes}',end='')
            sys.stdout.flush()
        episode = generate_episode(env)
        state ,actions,reward = zip(*episode)
        discount_rate = np.array([gamma ** i for i in range(len(reward)+ 1)])
        
        for s,state in enumerate(state):
            return_sum [state][action [s]] += sum(reward[s:] * discount_rate[:-(1+s)]) 
            N [state][action [s]] += 1.0
            Q [state][action [s]] += return_sum[state][action[s]] / N[state][action[s]]
        return Q 

Use the cell below to obtain the action-value function estimate $Q$.  We have also plotted the corresponding state-value function.

To check the accuracy of your implementation, compare the plot below to the corresponding plot in the solutions notebook **Monte_Carlo_Solution.ipynb**.

In [ ]:
# obtain the action-value function
Q = mc_prediction_q(env, 500000, generate_episode_from_limit_stochastic)

# obtain the corresponding state-value function
V_to_plot = dict((k,(k[0]>18)*(np.dot([0.8, 0.2],v)) + (k[0]<=18)*(np.dot([0.2, 0.8],v))) \
         for k, v in Q.items())

# plot the state-value function
plot_blackjack_values(V_to_plot)

### Part 2: MC Control

In this section, you will write your own implementation of constant-$\alpha$ MC control.  

Your algorithm has four arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.
- `policy`: This is a dictionary where `policy[s]` returns the action that the agent chooses after observing state `s`.

(_Feel free to define additional functions to help you to organize your code._)

In [ ]:
def generate_episodes_from_q(env,Q ,epsilon, nA):
    episode= []
    state = env.reset()
    while 1:
        action = np.random.choice(np.arange(nA),p = get_probs(Q[state],epsilon,nA))\
                                                if state in Q else env.action_space.samples()
        next_state,reward,done = evn.step(action)
        episode.append(state,action,reward)
        state = next_state
        if done:
            break
            
    return episode

def get_probs(Q_s , epsilon , nA):
    policy_s = np.ones(nA) * epsilon / (nA if nA != 0 else 1)
    best_a = np.argmax(Q_s)
    policy_s[best_a] = 1 - epsilon + (epsilon/nA)
    return policy_s

def update_Q (env, episode, Q, alpha, gamma):
    state,action,reward = zip(*episode)
    discounts = np.array([gamma**i for i in range(len(reward))])
    for i ,state in enumerate(state):
        old_Q = Q[state][action[i]]
        Q[state][action[i]] = old_Q + alpha*(sum(reward[i:] * discounts[:-(1+i)]) - old_Q )
    return Q   

In [ ]:
def mc_control(env, num_episodes, alpha, gamma=1.0,eps_start = 0.1,eps_decay =0.9999,eps_min = 0.05):
    nA = env.action_space.n
    # initialize empty dictionary of arrays
    Q = defaultdict(lambda: np.zeros(nA))
    # loop over episodes
    epsilon = eps_start
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        epsilon = max(epsilon * eps_decay , eps_min)
        episode = generate_episodes_from_q(env,Q , epsilon , nA)
        Q = update_Q (env, episode,Q, alpha,gamma)
    policy = dict((k,np.argmax(v)) for k,v in Q.items())
    return policy, Q

Use the cell below to obtain the estimated optimal policy and action-value function.  Note that you should fill in your own values for the `num_episodes` and `alpha` parameters.

In [ ]:
# obtain the estimated optimal policy and action-value function
policy, Q = mc_control(env, 500000, 0.02)

Next, we plot the corresponding state-value function.

In [ ]:
# obtain the corresponding state-value function
V = dict((k,np.max(v)) for k, v in Q.items())

# plot the state-value function
plot_blackjack_values(V)

Finally, we visualize the policy that is estimated to be optimal.

In [ ]:
# plot the policy
plot_policy(policy)

The **true** optimal policy $\pi_*$ can be found in Figure 5.2 of the [textbook](http://go.udacity.com/rl-textbook) (and appears below).  Compare your final estimate to the optimal policy - how close are you able to get?  If you are not happy with the performance of your algorithm, take the time to tweak the decay rate of $\epsilon$, change the value of $\alpha$, and/or run the algorithm for more episodes to attain better results.

![True Optimal Policy](images/optimal.png)